# Introduction à Langchain 



### Dépendances et clé d'API

In [3]:
#Décommenter sur Google Colab
#%pip install langchain langchain-openai langchain_mistralai openai python-dotenv -q
#from google.colab import userdata
#api_key=userdata.get('OPENAI_API_KEY')


#Décommenter en local
from dotenv import load_dotenv
from os import getenv
load_dotenv()
api_key= getenv("OPENAI_API_KEY")



## Vanilla via api OpenAI

Sortez les rames :)

In [13]:
from openai import OpenAI
from openai.types.responses import ResponseFunctionToolCall


client = OpenAI()


def initial_llm_request(user_message: str):
    """
    Make the initial request to the LLM with the user's message.
    """
    tools = [{
    "type": "function",
    "name": "get_weather",
    "description": "Get current temperature for a given location.",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "City and country e.g. Bogotá, Colombia"
            }
        },
        "required": [
            "location"
        ],
        "additionalProperties": False
    }
}]
    return client.responses.create(
        model="gpt-4o-mini",
        input=[{"role": "user", "content": user_message}],
        tools=tools
    )

def get_weather(location: str) -> str:
    """
    Simulate a weather API call.
    In a real-world scenario, you would replace this with an actual API call to a weather service.
    """
    # Simulated response
    return f"The current temperature in {location} is 20°C with clear skies."



def handle_tool_call(response):
    """
    Handle the tool call response from the LLM.
    If the response contains a tool call, execute it and return the result.
    """
    if isinstance(response.output[0], ResponseFunctionToolCall):
        tool_call = response.output[0]
        if tool_call.name == "get_weather":
            import json
            tool_args = json.loads(tool_call.arguments)
            location = tool_args.get("location", "")
            weather = get_weather(location)
            return weather
    return None


def final_llm_call(user_message: str, weather_info: str):
    """
    Make a final call to the LLM with the weather information.
    """
    response = client.responses.create(
        model="gpt-4o-mini",
        input=[{"role": "user", "content": user_message}, {"role": "developer", "content": weather_info}],
    )

    return response.output_text



user_message = "What is the weather like in Paris today?"
initial_response = initial_llm_request(user_message)
weather_info = handle_tool_call(initial_response)
if weather_info:
    final_response = final_llm_call(user_message, weather_info)
    print(final_response)
else:
    print("No tool call was made by the LLM.")


Today in Paris, the weather is clear with a temperature of 20°C. It sounds like a lovely day to explore the city!


## Langchain Tools et toolkits

In [ ]:
import os

from langchain_community.utilities import OpenWeatherMapAPIWrapper

os.environ["OPENWEATHERMAP_API_KEY"] = ""

weather = OpenWeatherMapAPIWrapper()
weather_data = weather.run("London,GB")
print(weather_data)

Nme: youtube_search
Description: search for youtube videos associated with a person. the input to this tool should be a comma separated list, the first part contains a person name and the second a number that is the maximum number of video results to return aka num_results. the second part is optional
Args: {'query': {'title': 'Query', 'type': 'string'}}


In [ ]:
import os

from langgraph.prebuilt import create_react_agent

os.environ["OPENAI_API_KEY"] = ""
os.environ["OPENWEATHERMAP_API_KEY"] = ""

tools = [weather.run]
agent = create_react_agent("openai:gpt-4.1-mini", tools)